In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define paths for Kaggle
train_dataset_path = '/kaggle/input/teeth-dataset/Teeth_Dataset/Training'
validation_dataset_path = '/kaggle/input/teeth-dataset/Teeth_Dataset/Validation'
test_dataset_path = '/kaggle/input/teeth-dataset/Teeth_Dataset/Testing'

# Image and augmentation parameters
image_size = (256, 256)  # ImageDataGenerator expects a 2D tuple for resizing
batch_size = 32

# Dictionary for folder-to-label mapping
folder_to_label = {'CaS': 0, 'CoS': 1, 'Gum': 2, 'MC': 3, 'OC': 4, 'OLP': 5, 'OT': 6}

# Initialize ImageDataGenerator for training (with augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# For validation and testing, we only rescale the images, no augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

# Generate augmented images and labels for training
train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=image_size,
    color_mode='rgb',
    classes=list(folder_to_label.keys()),
    class_mode='sparse',
    batch_size=batch_size,
    shuffle=True
)

# Generate images and labels for validation
validation_generator = test_datagen.flow_from_directory(
    validation_dataset_path,
    target_size=image_size,
    color_mode='rgb',
    classes=list(folder_to_label.keys()),
    class_mode='sparse',
    batch_size=batch_size,
    shuffle=False
)

# Generate images and labels for testing
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=image_size,
    color_mode='rgb',
    classes=list(folder_to_label.keys()),
    class_mode='sparse',
    batch_size=batch_size,
    shuffle=False
)

# Load ResNet50 with pre-trained weights, excluding top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add global average pooling layer
x = Dense(1024, activation='relu')(x)  # Add a fully connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(len(folder_to_label), activation='softmax')(x)  # Add output layer with softmax

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of ResNet50 so they are not trainable
for layer in base_model.layers[-10:]:  # Unfreeze the last 10 layers, for example
    layer.trainable = True
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)

# Train the model
model.fit(
    train_generator,
    epochs=100,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

# Optionally: Save the model
model.save('/kaggle/working/resnet50_teeth_classification.h5')


Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1724643691.688995     124 service.cc:145] XLA service 0x7c11e40051d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724643691.689046     124 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1724643691.689050     124 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1724643722.651529     124 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_28', 20 bytes spill stores, 20 bytes spill loads

I0000 00:00:1724643722.696

97/97 ━━━━━━━━━━━━━━━━━━━━ 156s 919ms/step - accuracy: 0.3523 - loss: 1.6798 - val_accuracy: 0.1556 - val_loss: 2.2250 - learning_rate: 1.0000e-04
Epoch 2/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 57s 552ms/step - accuracy: 0.8182 - loss: 0.5383 - val_accuracy: 0.1449 - val_loss: 2.5916 - learning_rate: 1.0000e-04
Epoch 3/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 57s 545ms/step - accuracy: 0.9112 - loss: 0.3069 - val_accuracy: 0.1158 - val_loss: 2.5695 - learning_rate: 1.0000e-04
Epoch 4/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 57s 551ms/step - accuracy: 0.9390 - loss: 0.1963 - val_accuracy: 0.1809 - val_loss: 2.4449 - learning_rate: 2.0000e-05
Epoch 5/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 57s 548ms/step - accuracy: 0.9787 - loss: 0.0765 - val_accuracy: 0.2062 - val_loss: 2.5210 - learning_rate: 2.0000e-05
Epoch 6/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 57s 543ms/step - accuracy: 0.9812 - loss: 0.0677 - val_accuracy: 0.2198 - val_loss: 2.5052 - learning_rate: 4.0000e-06
Epoch 7/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 57s 552ms/step - accurac

In [2]:
pip install streamlit


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 54.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


2024-08-26 04:06:56.380 
  command:

    streamlit run /opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://34.80.33.43:8501

